# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [3]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [48]:
def setup_data():
    
    """ reads in the data file, creates DayOfYear, Year and Temperature columns
    """
    
    # read the data
    df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
    
    # Convert Date column to date data type
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    
    # Create DayOfYear and year columns
    df['DayOfYear'] = df['Date'].dt.strftime('%m/%d')
    df['Year'] = df['Date'].dt.year
    
    # Get rid of February 29th dates
    df = df[df.DayOfYear != '02/29']
    
    # Sort by the Date column
    df = df.sort_values('Date')
    
    # Get temperature in Deg C
    df['Temperature'] = df['Data_Value']/10.0
    
    # Drop the Data_Value and Date columns
    for col in ['Data_Value', 'Date']:
        df = df.drop(col, axis=1)
    
    return df

setup_data().head()

,ID,Element,DayOfYear,Year,Temperature
18049,USW00014853,TMAX,01/01,2005,5.6
35479,USC00201502,TMIN,01/01,2005,-3.9
49823,USC00200228,TMAX,01/01,2005,15.0
17153,USC00207320,TMAX,01/01,2005,15.0
49827,USC00200228,TMIN,01/01,2005,-3.9


In [49]:
def get_highs_lows_no_2015():
    
    """
    Get the record highs and lows for years before 2015 and return a Data Frame
    """
    
    # Remove data from before 2015
    df = setup_data()
    df = df[df.Year < 2015]
    
    # Get the max and min based on DayOfYear
    result = df.groupby('DayOfYear')['Temperature'].agg(['max', 'min'])
    
    return result

get_highs_lows_no_2015()

,max,min
DayOfYear,,
01/01,15.6,-16.0
01/02,13.9,-26.7
01/03,13.3,-26.7
01/04,10.6,-26.1
01/05,12.8,-15.0
01/06,18.9,-26.6
01/07,21.7,-30.6
01/08,19.4,-29.4
01/09,17.8,-27.8


In [60]:
def get_highs_lows_2015():
    
    """
    Get the record highs and lows that occurred in 2015
    """
    
    pre2015 = get_highs_lows_no_2015()
    df = setup_data()
    df = df[df.Year == 2015]
        
    # Get the max and min based on DayOfYear in 2015
    df2015 = df.groupby('DayOfYear')['Temperature'].agg(['max', 'min']).rename(columns={'max': 'max2015', 'min': 'min2015'})
    
    # Merge the pre2015 and 2015 DataFrames
    result = pd.merge(pre2015, df2015, how='outer', left_index=True, right_index=True)
    
    # Determine if min or max is more extreme than pre2015
    result['min_record_in_2015'] = result['min2015'] < result['min']
    result['max_record_in_2015'] = result['max2015'] > result['max']
    
    # Remove the rows with both values False
    result = result[(result.min_record_in_2015 == True) | (result.max_record_in_2015 == True)]
    
    return result

get_highs_lows_2015()

,max,min,max2015,min2015,min_record_in_2015,max_record_in_2015
DayOfYear,,,,,,
01/05,12.8,-15.0,2.8,-15.5,True,False
01/11,15.6,-18.3,0.6,-20.0,True,False
02/03,8.9,-23.2,-2.1,-23.8,True,False
02/09,7.8,-21.0,8.3,-11.6,False,True
02/14,10.6,-21.7,-2.7,-23.9,True,False
02/15,11.7,-25.6,-3.9,-26.0,True,False
02/16,14.4,-22.2,-12.1,-29.4,True,False
02/17,15.6,-24.3,-5.5,-27.2,True,False
02/19,12.8,-18.8,-5.6,-26.0,True,False
